<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-29 11:20:14
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  10.00 K
Current:  1.33 C
-------------------
Today PnL: -18.20 K (-0.14%)
Current PnL: -18.49 L (-12.88%)
CY Booked + Current PnL: -10.41 L (-7.25%)
-------------------
Total profit:  2.51 L
Total loss:  -21.00 L
-------------------
Total Booked + Current PnL: 16.57 L (14.22%)
Total Booked PnL: 35.06 L (30.08%)
Curr Year Booked PnL: 8.08 L (6.09%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.09 C
Est FTT PnL: 76.70 L (57.81%)
Deployed:  1.17 C
Current:  1.33 C
CAGR/XIRR %: 8.37%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.00,-221.81,H-SC,12.13,87906.0,-7410.0,7446.0,1.31,-7.77,8.47,0.04,161.0,-1.00,0.66,73.38,OX40N,NTT,MISC
39,HINDALCO,651.95,761.55,-19.90,H-LC,5.38,110200.0,5888.0,11648.0,-0.09,5.64,10.57,16.81,10.0,0.51,0.83,22.60,X5K,ATH,METALS
4,APOLLOHOSP,6904.43,8285.00,-18.40,H-LC,6.68,111195.0,7629.0,13077.0,0.77,7.37,11.76,20.00,14.0,0.58,0.84,22.48,X40N,BTT,HEALTHCARE
17,BRITANNIA,5190.70,6446.05,12.14,H-LC,9.65,100692.0,7259.0,15335.0,-0.54,7.77,15.23,24.18,35.0,0.47,0.76,22.27,XY25,ATH,FMCG
58,MEDANTA,1087.93,1486.00,21.15,H-SC,11.27,155278.0,29078.0,17096.0,-1.24,23.04,11.01,36.59,154.0,1.70,1.17,38.44,XY24,NTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,QUESS,296.83,986.00,-34.44,M-SC,42.09,65196.0,190.0,150740.0,1.29,0.29,231.21,232.18,196.0,0.00,0.49,1.29,XY24,NTT,MISC
14,BERGEPAINT,561.33,680.00,-19.92,H-MC,10.81,226476.0,-863.0,48919.0,-0.16,-0.38,21.60,21.14,108.0,-0.02,1.71,26.80,XY24,NTT,PAINTS
38,HEROMOTOCO,4311.81,5949.07,-6.19,H-MC,3.15,151270.0,357.0,56953.0,0.72,0.24,37.65,37.97,89.0,0.01,1.14,25.60,AR,ATH,AUTO
59,NATIONALUM,189.63,247.44,-50.15,H-MC,3.55,100128.0,-376.0,31020.0,0.65,-0.37,30.98,30.49,81.0,-0.01,0.76,31.91,MH,ATH,METALS


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,CIPLA,1492.70,1795.0,-18.42,H-LC,1.36,214994.0,10494.0,30916.0,-0.17,5.13,14.38,20.25,7.0,0.34,1.62,14.31,X40N,BTT,PHARMA
56,LTIM,5564.16,7230.2,-5.37,H-LC,2.55,181962.0,-18348.0,78316.0,0.15,-9.16,43.04,29.94,26.0,-0.23,1.37,25.71,X200,ATH,IT
10,BAJAJHFL,122.26,152.0,-8.77,H-LC,2.76,189207.0,-11299.0,60073.0,0.15,-5.64,31.75,24.33,30.0,-0.19,1.43,9.11,X40N,BTT,FINANCE
40,HINDUNILVR,2413.81,2723.0,-12.82,H-LC,3.39,272832.0,2485.0,32140.0,-0.23,0.92,11.78,12.81,18.0,0.08,2.06,14.10,XY25,NTT,FMCG
5,ASIANPAINT,2961.56,4250.0,-19.77,H-LC,3.43,159304.0,-42082.0,129689.0,-0.72,-20.90,81.41,43.51,20.0,-0.32,1.20,10.89,X40,BTT,PAINTS


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,BSOFT,628.85,836.99,-17.09,M-SC,12.41,69528.0,-40521.0,76947.0,3.13,-36.82,110.67,33.10,216.0,-0.53,0.52,15.53,XR,ATH,IT
51,JIOFIN,310.58,387.00,3.13,H-LC,13.23,221025.0,3619.0,49885.0,2.73,1.66,22.57,24.61,49.0,0.07,1.67,57.15,XY24,BTT,FINANCE
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,20.60,67968.0,-45581.0,85633.0,2.08,-40.14,125.99,35.27,267.0,-0.53,0.51,66.35,XR,NTT,HOTELS
53,KPIGREEN,497.21,731.64,25.50,H-SC,12.65,128344.0,3047.0,56035.0,1.70,2.43,43.66,47.15,143.0,0.05,0.97,60.57,MH,ATH,POWER
79,TANLA,917.30,1963.11,-33.71,H-SC,6.56,162603.0,-76812.0,349775.0,1.44,-32.08,215.11,114.01,130.0,-0.22,1.23,49.13,AR,ATH,IT


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
90,VALIANTORG,512.64,838.0,-316.47,H-SC,21.18,92040.0,-41246.0,125837.0,-2.79,-30.95,136.72,63.47,150.0,-0.33,0.69,54.28,XR,NTT,CHEMICALS
31,EASEMYTRIP,18.12,26.4,1.39,M-SC,19.79,97580.0,-79235.0,160031.0,-2.25,-44.81,164.00,45.70,195.0,-0.50,0.74,0.00,XY24,NTT,TRAVEL
21,CERA,7989.07,8422.0,-18.85,H-SC,2.92,116010.0,-27793.0,35580.0,-1.48,-19.33,30.67,5.42,155.0,-0.78,0.88,28.02,OX40N,NTT,CERAMICS
19,CAMPUS,294.86,393.0,-26.47,M-SC,3.00,153175.0,-14600.0,70445.0,-1.48,-8.70,45.99,33.28,208.0,-0.21,1.16,18.59,XY24,NTT,FOOTWEAR
9,BAJAJFINSV,1994.80,2758.0,-4.26,H-LC,8.92,95516.0,-2229.0,39630.0,-1.44,-2.28,41.49,38.26,41.0,-0.06,0.72,27.54,X40,BTT,FINANCE


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.0,-221.81,H-SC,12.13,87906.0,-7410.0,7446.0,1.31,-7.77,8.47,0.04,161.0,-1.00,0.66,73.38,OX40N,NTT,MISC
78,SYMPHONY,1306.42,1306.0,-31.84,M-SC,9.54,145410.0,-25731.0,25679.0,0.11,-15.03,17.66,-0.03,194.0,-1.00,1.10,5.23,OX40N,NTT,DURABLES
86,UJJIVANSFB,52.77,53.0,35.36,M-SC,21.72,117558.0,-24921.0,25545.0,-0.57,-17.49,21.73,0.44,249.0,-0.98,0.89,38.75,OX40N,NTT,BANKS
85,TTKPRESTIG,769.29,770.0,-9.57,H-SC,12.04,82661.0,-18116.0,18210.0,-0.93,-17.98,22.03,0.09,144.0,-0.99,0.62,7.57,OX40N,NTT,DURABLES
80,TATAELXSI,7332.28,7332.0,-19.83,H-MC,1.73,84154.0,-18498.0,18497.0,-0.41,-18.02,21.98,-0.00,100.0,-1.00,0.63,27.92,OX40N,NTT,IT


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3176.70,NaN,NaN,2.46,74490.0,40041.0,4924.0,-1.25,116.23,6.61,130.54,1.0,8.13,0.56,13.91,AR,ATH,ELECTRICAL
91,VBL,492.64,672.28,-13.02,H-LC,12.52,264077.0,-963.0,97603.0,0.84,-0.36,36.96,36.46,2.0,-0.01,1.99,12.78,X40N,ATH,FMCG
82,TCS,3794.03,4998.00,-27.06,H-LC,12.75,238040.0,-57894.0,151798.0,-0.89,-19.56,63.77,31.73,3.0,-0.38,1.80,0.00,X40,BTT,IT
48,INFY,1461.46,2275.00,-16.01,H-LC,6.81,269699.0,8098.0,137520.0,-0.61,3.10,50.99,55.67,5.0,0.06,2.03,9.39,X40,BTT,IT
1,ABB,5551.76,7934.00,-39.96,H-LC,7.44,214442.0,-2077.0,94976.0,-0.85,-0.96,44.29,42.91,6.0,-0.02,1.62,12.09,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,267.99,NaN,NaN,3.21,52753.0,11498.0,5935.0,1.12,27.87,11.25,42.26,197.0,1.94,0.40,24.20,XY24,ATH,IT
58,MEDANTA,1087.93,1486.00,21.15,H-SC,11.27,155278.0,29078.0,17096.0,-1.24,23.04,11.01,36.59,154.0,1.70,1.17,38.44,XY24,NTT,HEALTHCARE
32,ENRIN,1377.95,3176.70,NaN,NaN,2.46,74490.0,40041.0,4924.0,-1.25,116.23,6.61,130.54,1.0,8.13,0.56,13.91,AR,ATH,ELECTRICAL


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,KPIGREEN,497.21,731.64,25.50,H-SC,12.65,128344.0,3047.0,56035.0,1.70,2.43,43.66,47.15,143.0,0.05,0.97,60.57,MH,ATH,POWER
58,MEDANTA,1087.93,1486.00,21.15,H-SC,11.27,155278.0,29078.0,17096.0,-1.24,23.04,11.01,36.59,154.0,1.70,1.17,38.44,XY24,NTT,HEALTHCARE
20,CAMS,3643.00,5226.82,-2.58,H-SC,0.72,109542.0,7538.0,36806.0,0.41,7.39,33.60,43.48,122.0,0.20,0.83,25.41,X40N,ATH,MISC
38,HEROMOTOCO,4311.81,5949.07,-6.19,H-MC,3.15,151270.0,357.0,56953.0,0.72,0.24,37.65,37.97,89.0,0.01,1.14,25.60,AR,ATH,AUTO
26,DABUR,505.20,735.00,-7.00,H-MC,4.50,201221.0,4193.0,85438.0,-1.16,2.13,42.46,45.49,96.0,0.05,1.52,15.45,XY24,BTT,FMCG


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3176.70,NaN,NaN,2.46,74490.0,40041.0,4924.0,-1.25,116.23,6.61,130.54,1.0,8.13,0.56,13.91,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1219.98,2270.00,-37.93,M-SC,8.40,104325.0,12826.0,65923.0,0.82,14.02,63.19,86.07,205.0,0.19,0.79,50.92,XR,NTT,DURABLES
45,INDIAMART,2327.09,4850.92,-48.45,H-SC,12.79,134254.0,10918.0,122842.0,-0.91,8.85,91.50,108.45,118.0,0.09,1.01,33.18,AR,ATH,MISC
53,KPIGREEN,497.21,731.64,25.50,H-SC,12.65,128344.0,3047.0,56035.0,1.70,2.43,43.66,47.15,143.0,0.05,0.97,60.57,MH,ATH,POWER
95,WIPRO,243.46,420.00,-9.67,M-LC,5.65,154907.0,3962.0,105492.0,-0.08,2.62,68.10,72.51,56.0,0.04,1.17,8.81,XR,NTT,IT


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,94.95,NaN,NaN,6.57,19789.0,-29804.0,1005.0,1.27,-60.10,5.08,-58.07,204.0,-29.66,0.15,32.71,XY24,ATH,MISC
74,SIS,477.00,477.00,2275.65,M-SC,5.80,63920.0,-17170.0,17169.0,0.20,-21.17,26.86,0.00,238.0,-1.00,0.48,29.54,OX40N,NTT,MISC
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,20.60,67968.0,-45581.0,85633.0,2.08,-40.14,125.99,35.27,267.0,-0.53,0.51,66.35,XR,NTT,HOTELS
92,VIPIND,488.80,489.00,-221.81,H-SC,12.13,87906.0,-7410.0,7446.0,1.31,-7.77,8.47,0.04,161.0,-1.00,0.66,73.38,OX40N,NTT,MISC
57,MASFIN,326.60,399.50,-17.34,H-SC,16.82,93840.0,-4140.0,26012.0,-1.31,-4.23,27.72,22.32,149.0,-0.16,0.71,36.10,XR,ATH,FINANCE


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,267.99,NaN,NaN,3.21,52753.0,11498.0,5935.0,1.12,27.87,11.25,42.26,197.0,1.94,0.40,24.20,XY24,ATH,IT
33,GREENPANEL,375.16,537.00,250.78,M-SC,9.43,156550.0,-26528.0,105499.0,0.17,-14.49,67.39,43.14,234.0,-0.25,1.18,47.26,XY24,NTT,MISC
16,BLUSPRING,226.45,94.95,NaN,NaN,6.57,19789.0,-29804.0,1005.0,1.27,-60.10,5.08,-58.07,204.0,-29.66,0.15,32.71,XY24,ATH,MISC
58,MEDANTA,1087.93,1486.00,21.15,H-SC,11.27,155278.0,29078.0,17096.0,-1.24,23.04,11.01,36.59,154.0,1.70,1.17,38.44,XY24,NTT,HEALTHCARE
72,SHALBY,261.39,327.00,971.77,M-SC,9.65,145696.0,-35447.0,80920.0,0.29,-19.57,55.54,25.10,228.0,-0.44,1.10,16.76,XY24,NTT,HEALTHCARE


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.52
1,25,41.29
2,50,70.21


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    41.66
LC    35.24
MC    22.07
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.85
X40N     14.65
X40      12.53
XY25     10.80
XR        9.11
OX40N     8.40
AR        8.23
X5K       2.25
MH        1.73
X200      1.37
SR        1.16
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    26.99
H-SC    25.21
H-MC    19.08
M-SC    15.04
M-LC     7.19
M-MC     2.64
L-SC     1.41
L-LC     1.06
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.17,-4.23,39.10
IT,11.76,-16.60,74.04
FINANCE,8.33,-18.14,61.01
BANKS,7.54,-14.13,60.70
MISC,6.09,-18.99,74.56
PAINTS,5.63,-17.06,40.13
ELECTRICAL,5.39,-3.27,52.20
HEALTHCARE,4.66,2.23,23.74
AUTO,4.44,-14.84,64.80


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2761819.0,25
XR,1087543.0,13
AR,1055258.0,10
X40,792873.0,10
X40N,608991.0,12
XY25,507568.0,8
OX40N,359858.0,11
SR,190253.0,2
X5K,140566.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2481333.0,26
M-SC,1640109.0,20
H-LC,1288264.0,20
H-MC,1177730.0,15
M-LC,424291.0,5
M-MC,328416.0,2
L-SC,212834.0,3
L-MC,56526.0,1
L-LC,48733.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,964391.0,7
M-SC,XY24,783993.0,7
H-SC,AR,526493.0,3
H-LC,X40,513754.0,6
H-MC,XY24,455995.0,4
H-SC,XR,365397.0,4
M-MC,XY24,328416.0,2
H-LC,X40N,307539.0,6
M-SC,XR,286662.0,4


In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
